# Example for utilization of powerfactory-tools

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib

from powerfactory_tools.exporter import PowerfactoryExporter
from powerfactory_tools.exporter import export_powerfactory_data
from powerfactory_tools.schema.steadystate_case.case import Case as SteadystateCase
from powerfactory_tools.schema.topology.topology import Topology
from powerfactory_tools.schema.topology_case.case import Case as TopologyCase

In [ ]:
PROJECT_NAME = "PowerFactory-Tools"  # may be also full path "dir_name\project_name"
GRID_NAME = "HV_8_Bus"
EXPORT_PATH = pathlib.Path("export")
PF_USER_PROFILE = ""    # specification may be necessary

## Export using an exporter instance

In [ ]:
with PowerfactoryExporter(project_name=PROJECT_NAME, grid_name=GRID_NAME, powerfactory_user_profile=PF_USER_PROFILE) as exporter:
    # for this example switch to base scenario
    exporter.switch_scenario(scen="base")
    # exporter will generate now three JSON files: topology, topology_case, steadystate_case
    # topology_name is set by default to "GRID_NAME_topology.json"
    exporter.export(EXPORT_PATH, topology_name=f"{GRID_NAME}", topology_case_name=f"{GRID_NAME}", steadystate_case_name=f"{GRID_NAME}")

    # switch to scenario "OutOfService_Set1" and export topology_case and steadystate_case (assuming that topology keeps the same)
    exporter.export_scenario(export_path=EXPORT_PATH, scenario_name="outOfService_Set1", topology_case_name=f"{GRID_NAME}_Set1", steadystate_case_name=f"{GRID_NAME}_Set1")

## Export using the export function running in a new process with default parameters

In [ ]:
# As the export function is executed in a process that is terminated after execution, the PowerFactory API is also closed.
export_powerfactory_data(export_path=EXPORT_PATH, project_name=PROJECT_NAME, grid_name=GRID_NAME, powerfactory_user_profile=PF_USER_PROFILE)

### [Optional] Check JSON Import and display exported JSON

In [ ]:
t = Topology.from_file(EXPORT_PATH / f"{GRID_NAME}_topology.json")
js = t.json(sort_keys=True)
t2 = Topology.from_json(js)
assert t2 == t

In [ ]:
print(t2.json(indent=2))

In [ ]:
tc = TopologyCase.from_file(EXPORT_PATH / f"{GRID_NAME}_topology_case.json")
js2 = tc.json(sort_keys=True)
tc2 = TopologyCase.from_json(js2)
assert tc2 == tc

In [ ]:
print(tc2.json(indent=2))

In [ ]:
ssc = SteadystateCase.from_file(EXPORT_PATH / f"{GRID_NAME}_steadystate_case.json")
js2 = ssc.json(sort_keys=True)
ssc2 = SteadystateCase.from_json(js2)
assert ssc2 == ssc

In [ ]:
print(ssc2.json(indent=2))

### Display raw JSON schema

In [ ]:
print(Topology.schema_json(indent=2))

In [ ]:
print(TopologyCase.schema_json(indent=2))

In [ ]:
print(SteadystateCase.schema_json(indent=2))